# Working with ISMI project dates

## Load date samples from RDF

In [ ]:
from rdflib import Graph, RDF, URIRef
from rdflib.namespace import Namespace, RDFS
from undate.undate import Undate

# additional RDF namespaces
crmNs = Namespace('http://www.cidoc-crm.org/cidoc-crm/')

g = Graph()
g.bind('crm', crmNs)
# load ISMI RDF sample data
g.parse('data/ismi-crm-date-samples.ttl')
# check: number of triples
len(g)

In [ ]:
date_uris = [u for u in g.subjects(RDF.type, crmNs['E52_Time-Span'])]

for uri in date_uris:
    q = '''SELECT ?uri ?label ?note 
           WHERE { 
             ?uri crm:P3_has_note ?note ;
               crm:P1_is_identified_by / rdfs:label ?label .
            } limit 10'''
    res = g.query(q, initBindings={'uri': uri})
    for r in res:
        print(f"uri={str(uri)} label={r.label} note={r.note}")

## Convert RDF dates to Undate

In [ ]:
from undate.date import DatePrecision, Date
import datetime

uri = date_uris[1]

#
# read date type
#
date_type = None
for date_type_uri in g.objects(uri, crmNs.P2_has_type):
    for dt in ['day', 'year', 'range']:
        if str(date_type_uri) == 'http://content.mpiwg-berlin.mpg.de/ns/ismi/type/date/' + dt:
            date_type = dt

if not date_type:
    raise RuntimeError(f"Unknown datetype URI {date_type_uri}")

#
# read label and calendar
#
date_label_uri = next(g.objects(uri, crmNs.P1_is_identified_by))
date_label = str(next(g.objects(date_label_uri, RDFS.label)))
for date_label_calendar_uri in g.objects(date_label_uri, crmNs.P2_has_type):
    for ct in ['gregorian', 'julian', 'islamic']:
        if str(date_label_calendar_uri) == 'http://content.mpiwg-berlin.mpg.de/ns/ismi/type/calendar/' + ct:
            calendar_type = ct

if not calendar_type:
    raise RuntimeError(f"Unknown calendar type URI {date_label_calendar_uri}")

#
# create undate
#
if date_type == 'day':
    xsd_date = next(g.objects(uri, crmNs.P82_at_some_time_within))
    date = Undate.parse(str(xsd_date), 'ISO8601')
    date.precision = DatePrecision.DAY
    date.label = date_label

if date_type == 'year':
    xsd_date_from = next(g.objects(uri, crmNs.P82a_begin_of_the_begin))
    xsd_date_until = next(g.objects(uri, crmNs.P82b_end_of_the_end))
    date_from = datetime.date.fromisoformat(str(xsd_date_from))
    if calendar_type == 'gregorian':
        # this should be fine
        date = Undate(year=date_from.year)

    else:
        # create day precision Undate from end date
        date = Undate.parse(str(xsd_date_until), 'ISO8601')
        # change earliest date
        date.earliest = Date(year=date_from.year, month=date_from.month, day=date_from.day)

    # change precision and label
    date.precision = DatePrecision.DAY
    date.label = date_label

if date_type == 'range':
    xsd_date_from = next(g.objects(uri, crmNs.P82a_begin_of_the_begin))
    xsd_date_until = next(g.objects(uri, crmNs.P82b_end_of_the_end))
    # create day precision Undate from start date
    date = Undate.parse(str(xsd_date_from), 'ISO8601')
    # change latest date
    date_until = datetime.date.fromisoformat(str(xsd_date_until))
    date.latest = Date(year=date_until.year, month=date_until.month, day=date_until.day)
    # change precision and label
    date.precision = DatePrecision.DAY
    date.label = date_label


print(f"{date_label=} {date_type=} {calendar_type=} {date=}")